In [1]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,True,False,False,False,True
9996,516,35,10,57369.61,1,1,1,101699.77,0,True,False,False,False,True
9997,709,36,7,0.00,1,0,1,42085.58,1,True,False,False,True,False
9998,772,42,3,75075.31,2,1,0,92888.52,1,False,True,False,False,True


In [8]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [9]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [12]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.7890 - accuracy: 0.5142
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.7874 - accuracy: 0.5149
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.7859 - accuracy: 0.5157
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7844 - accuracy: 0.5166
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7829 - accuracy: 0.5188
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7813 - accuracy: 0.5195
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7798 - accuracy: 0.5209
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7782 - accuracy: 0.5221
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.7766 - accuracy: 0.5235
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7750 - accura

In [13]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 1ms/step


array([[False],
       [False],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [14]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.84      0.66      0.74      1585
           1       0.29      0.52      0.37       415

    accuracy                           0.63      2000
   macro avg       0.57      0.59      0.56      2000
weighted avg       0.73      0.63      0.67      2000



In [15]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 0s 1ms/step - loss: 0.6573 - accuracy: 0.6241


[0.6573395133018494, 0.6241250038146973]

# Hyperparameter tuning


In [16]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [18]:
# This function optimizes No. of layers, No. of neurons in each layer, activation function and optimizer
def hyper(hp): #This defines a function named hyper that takes a single argument hp.
    model = Sequential()
    counter = 0 #Initializes a counter variable to keep track of the current layer being added to the model.
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):  # Find No. of hiddern layers: Loops through a range of layers,
        # where the number of layers is a hyperparameter named 'num_layers' with a minimum value of 1 and a maximum value of 10.
        # No. of Hidden layers are 1 to 10. In each layer need to add no. of neurons and activation function. So following if-else code
        if counter == 0: # If block to find no. of hidden layers, will be executed only once
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
            #Adds the first layer with input dimensions specified by input_dim=13.
            #The number of units ('units') and activation function ('activation') are hyperparameters,
            #and their values are determined by the hp object.
        else: # else block to find no of neurons and activation function
          model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))

              #Adds subsequent layers (after the first layer) with hyperparameter-specified units and activation functions.

    counter+=1   #Increments the counter to keep track of the layers added.
    model.add(Dense(units = 1,activation='sigmoid'))   #Adds the output layer with a single unit and sigmoid activation function, typical for binary classification problems.
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model  #Returns the compiled model.

In [19]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [20]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))
# search() is similar to fit()

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.8554999828338623

Best val_accuracy So Far: 0.8610000014305115
Total elapsed time: 00h 00m 20s


In [21]:
tuner.get_best_hyperparameters()[0].values # will give first model where accuracy is high

{'num_layers': 5,
 'units0': 32,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 104,
 'activation2': 'tanh',
 'units3': 8,
 'activation3': 'tanh',
 'units4': 64,
 'activation4': 'relu',
 'units5': 48,
 'activation5': 'sigmoid',
 'units6': 72,
 'activation6': 'tanh'}

In [22]:
tuned_model = tuner.get_best_models(num_models=1)[0]
# tuned_model.build(x.shape)

In [23]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 2s 3ms/step - loss: 0.3430 - accuracy: 0.8583 - val_loss: 0.3428 - val_accuracy: 0.8615
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3378 - accuracy: 0.8595 - val_loss: 0.3424 - val_accuracy: 0.8600
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3347 - accuracy: 0.8574 - val_loss: 0.3368 - val_accuracy: 0.8600
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3345 - accuracy: 0.8608 - val_loss: 0.3356 - val_accuracy: 0.8595
Epoch 5/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3314 - accuracy: 0.8614 - val_loss: 0.3340 - val_accuracy: 0.8625
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3290 - accuracy: 0.8589 - val_loss: 0.3371 - val_accuracy: 0.8590
Epoch 7/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3272 - accuracy: 0.8650 - val_loss: 0.3377 - val_accuracy: 0.8620
Epoch 

In [24]:
tuned_model.evaluate(xtrain,ytrain) # Training accuracy

250/250 [==============================] - 1s 2ms/step - loss: 0.1771 - accuracy: 0.9329


[0.1770702749490738, 0.9328749775886536]

In [25]:
tuned_model.evaluate(xtest,ytest)# Testing accuracy

63/63 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.8420


[0.4541604816913605, 0.8420000076293945]